In [2]:
from pyspark.sql import SparkSession

In [3]:
from pyspark.sql.functions import *
from pyspark import broadcast
from pyspark import StorageLevel
from pyspark.sql.types import *

In [6]:
spark = SparkSession.builder.appName("assessment").getOrCreate()
print(spark)

In [17]:
x_df = spark.read.csv("file:///home/hdoop/notebooks/X.csv",header =True,inferSchema = True)

In [18]:
y_df = spark.read.csv("file:///home/hdoop/notebooks/Y.csv",header =True,inferSchema =True)

## 2.2


In [20]:
x_df.printSchema()

root
 |-- invoice_no: integer (nullable = true)
 |-- gstin: string (nullable = true)
 |-- supplier_gstin: string (nullable = true)
 |-- invoice_date: date (nullable = true)
 |-- amount: integer (nullable = true)
 |-- gst: double (nullable = true)



In [32]:
x_df1 = x_df.withColumn("amount",when(isnull(col("amount")),10).otherwise(col("amount")))

In [33]:
y_df.printSchema()

root
 |-- invoice_no: integer (nullable = true)
 |-- gstin: string (nullable = true)
 |-- supplier_gstin: string (nullable = true)
 |-- invoice_date: date (nullable = true)
 |-- amount: integer (nullable = true)
 |-- gst: double (nullable = true)



In [34]:
y_df1 = y_df.withColumn("amount",when(isnull(col("amount")),10).otherwise(col("amount")))

In [35]:
x_df1.show()

+----------+-----+--------------+------------+------+---+
|invoice_no|gstin|supplier_gstin|invoice_date|amount|gst|
+----------+-----+--------------+------------+------+---+
|         3|    C|            a3|  2024-01-05|    12|2.4|
|         4|    D|            a4|  2024-01-06|    13|2.6|
|         5|    E|            a5|  2024-01-07|    14|2.8|
|         6|    F|            a6|  2024-01-08|    15|3.0|
|         7|    G|            a7|  2024-01-09|    16|3.2|
|         8|    H|            a8|  2024-01-10|    17|3.4|
|         9|    I|            a9|  2024-01-11|    18|3.6|
|        10|    J|           a10|  2024-01-12|    19|3.8|
|        11|    K|           a11|  2024-01-13|    20|4.0|
|        12|    L|           a12|  2024-01-14|    21|4.2|
|        13|    M|           a13|  2024-01-15|    22|4.4|
|        14|    N|           a14|  2024-01-16|    23|4.6|
|        15|    O|           a15|  2024-01-17|    24|4.8|
|        16|    P|           a16|  2024-01-18|    25|5.0|
|        17|  

In [36]:
x_df2 = x_df1.filter(isnotnull(col("invoice_no")))\
    .filter(isnotnull(col("gstin")))\
    .filter(isnotnull(col("supplier_gstin")))\
    .filter(isnotnull(col("invoice_date")))

In [37]:
y_df2 = y_df1.filter(isnotnull(col("invoice_no")))\
    .filter(isnotnull(col("gstin")))\
    .filter(isnotnull(col("supplier_gstin")))\
    .filter(isnotnull(col("invoice_date")))

In [38]:
x_df2.show()

+----------+-----+--------------+------------+------+---+
|invoice_no|gstin|supplier_gstin|invoice_date|amount|gst|
+----------+-----+--------------+------------+------+---+
|         3|    C|            a3|  2024-01-05|    12|2.4|
|         4|    D|            a4|  2024-01-06|    13|2.6|
|         5|    E|            a5|  2024-01-07|    14|2.8|
|         6|    F|            a6|  2024-01-08|    15|3.0|
|         7|    G|            a7|  2024-01-09|    16|3.2|
|         8|    H|            a8|  2024-01-10|    17|3.4|
|         9|    I|            a9|  2024-01-11|    18|3.6|
|        10|    J|           a10|  2024-01-12|    19|3.8|
|        11|    K|           a11|  2024-01-13|    20|4.0|
|        12|    L|           a12|  2024-01-14|    21|4.2|
|        13|    M|           a13|  2024-01-15|    22|4.4|
|        14|    N|           a14|  2024-01-16|    23|4.6|
|        15|    O|           a15|  2024-01-17|    24|4.8|
|        16|    P|           a16|  2024-01-18|    25|5.0|
|        17|  

In [45]:
x_df3 = x_df2.groupBy([col("invoice_no"),col("gstin"),col("supplier_gstin"),col("invoice_date")])\
    .agg(
        sum("amount").alias("amount"),
        sum("gst").alias("gst")
    )

In [46]:
y_df3 = y_df2.groupBy([col("invoice_no"),col("gstin"),col("supplier_gstin"),col("invoice_date")])\
    .agg(
        sum("amount").alias("amount"),
        sum("gst").alias("gst")
    )

#### 2.3

In [47]:
x_df3.printSchema()

root
 |-- invoice_no: integer (nullable = true)
 |-- gstin: string (nullable = true)
 |-- supplier_gstin: string (nullable = true)
 |-- invoice_date: date (nullable = true)
 |-- amount: long (nullable = true)
 |-- gst: double (nullable = true)



In [49]:
x_df4= x_df3.withColumn("total_amount",col("amount")+col("gst"))

In [50]:
y_df4= y_df3.withColumn("total_amount",col("amount")+col("gst"))


In [135]:
x_df4.printSchema()

root
 |-- invoice_no: integer (nullable = true)
 |-- gstin: string (nullable = true)
 |-- supplier_gstin: string (nullable = true)
 |-- invoice_date: date (nullable = true)
 |-- amount: long (nullable = true)
 |-- gst: double (nullable = true)
 |-- total_amount: double (nullable = true)



#### 2.4

In [53]:
x_df4.printSchema()

root
 |-- invoice_no: integer (nullable = true)
 |-- gstin: string (nullable = true)
 |-- supplier_gstin: string (nullable = true)
 |-- invoice_date: date (nullable = true)
 |-- amount: long (nullable = true)
 |-- gst: double (nullable = true)
 |-- total_amount: double (nullable = true)



In [58]:
x_df5 = x_df4.groupby([col("gstin"),col("supplier_gstin"),col("invoice_date")])\
 .agg(
     sum("amount").alias("amount"),
     sum("gst").alias("gst"),
     sum("total_amount").alias("total_amount")
 )

In [59]:
y_df5 = y_df4.groupby([col("gstin"),col("supplier_gstin"),col("invoice_date")])\
 .agg(
     sum("amount").alias("amount"),
     sum("gst").alias("gst"),
     sum("total_amount").alias("total_amount")
 )

### 2.5

In [81]:
@udf
def run_length(string_entered):
    marker = string_entered[0]
    counter = 0
    return_str = ""
    for char in string_entered:
        if (marker == char):
            counter +=1
        else:
            str = f"{marker}{counter}"
            return_str =return_str + str
            marker = char
            counter = 1
    str = f"{marker}{counter}"
    return_str =return_str + str
    return return_str

In [79]:
# run_length_udf = udf(run_length,StringType())

In [83]:
x_df6= x_df5.withColumn("supplier_gstin",run_length(col("supplier_gstin")))

In [84]:
y_df6= y_df5.withColumn("supplier_gstin",run_length(col("supplier_gstin")))

In [86]:
x_df7 = x_df6.withColumn("supplier_gstin",md5(col("supplier_gstin")))

In [87]:
y_df7 = y_df6.withColumn("supplier_gstin",md5(col("supplier_gstin")))

In [133]:
x_df7.show()

+-----+--------------------+------------+------+---+------------+
|gstin|      supplier_gstin|invoice_date|amount|gst|total_amount|
+-----+--------------------+------------+------+---+------------+
|    S|7b42e594c6b0aea17...|  2024-01-21|    28|5.6|        33.6|
|    L|04a6891d4d3f105be...|  2024-01-14|    21|4.2|        25.2|
|    K|d8715694717e03d6c...|  2024-01-13|    20|4.0|        24.0|
|    F|e4fde4e670403765f...|  2024-01-08|    15|3.0|        18.0|
|    H|ce106d4e16eb0fb67...|  2024-01-10|    17|3.4|        20.4|
|    N|cdb8baed319712f77...|  2024-01-16|    23|4.6|        27.6|
|    R|f5cd71aba90b526f8...|  2024-01-20|    27|5.4|        32.4|
|    U|73d50a19965eba5f9...|  2024-01-23|    30|6.0|        36.0|
|    C|3e4f4af15ee8fb8da...|  2024-01-05|    12|2.4|        14.4|
|    J|130ab96c963eada95...|  2024-01-12|    19|3.8|        22.8|
|    D|5496fa050b66ec447...|  2024-01-06|    13|2.6|        15.6|
|    Z|5520d55f74db52f7e...|  2024-01-28|    35|7.0|        42.0|
|    X|574

### 2.6

In [104]:
type_A = x_df4.join(y_df4,["invoice_no","gstin","supplier_gstin","invoice_date","amount"],"inner").select("invoice_no","gstin","supplier_gstin","invoice_date")
type_A = type_A.withColumn("category",lit("Type A"))

In [105]:
type_A.printSchema()

root
 |-- invoice_no: integer (nullable = true)
 |-- gstin: string (nullable = true)
 |-- supplier_gstin: string (nullable = true)
 |-- invoice_date: date (nullable = true)
 |-- category: string (nullable = false)



In [107]:
type_e = x_df4.join(y_df4,["invoice_no","gstin","supplier_gstin","amount"],"leftanti").select("invoice_no","gstin","supplier_gstin","invoice_date")
type_e = type_e.withColumn("category",lit("Type E"))

In [108]:
type_d = y_df4.join(x_df4,["invoice_no","gstin","supplier_gstin","amount"],"leftanti").select("invoice_no","gstin","supplier_gstin","invoice_date")
type_d = type_d.withColumn("category",lit("Type D"))

In [109]:
type_b = x_df4.join(y_df4,
                    (x_df4["invoice_no"]==y_df4["invoice_no"])&(x_df4["gstin"]==y_df4["gstin"])&(x_df4["supplier_gstin"]==y_df4["supplier_gstin"])&(x_df4["invoice_date"]==y_df4["invoice_date"])&( (x_df4["amount"]-y_df4["amount"]<=50) | (x_df4["amount"]-y_df4["amount"]>=-50)),
                    "inner").select(x_df4["invoice_no"],x_df4["gstin"],x_df4["supplier_gstin"],x_df4["invoice_date"])
type_b = type_b.withColumn("category",lit("Type B"))

In [110]:

type_c = x_df4.join(y_df4,
                    (x_df4["invoice_no"]==y_df4["invoice_no"])&(x_df4["gstin"]==y_df4["gstin"])&(x_df4["supplier_gstin"]==y_df4["supplier_gstin"])&(x_df4["invoice_date"]==y_df4["invoice_date"])&( (x_df4["amount"]-y_df4["amount"]>50) | (x_df4["amount"]-y_df4["amount"]<-50)),
                    "inner").select(x_df4["invoice_no"],x_df4["gstin"],x_df4["supplier_gstin"],x_df4["invoice_date"])
type_c = type_c.withColumn("category",lit("Type C"))

In [111]:
unioned_df = type_A.union(type_b).union(type_c).union(type_d).union(type_e)

In [112]:
unioned_df.show()

+----------+-----+--------------+------------+--------+
|invoice_no|gstin|supplier_gstin|invoice_date|category|
+----------+-----+--------------+------------+--------+
|         4|    D|            a4|  2024-01-06|  Type A|
|        13|    M|           a13|  2024-01-15|  Type A|
|         6|    F|            a6|  2024-01-08|  Type B|
|        18|    R|           a18|  2024-01-20|  Type B|
|        11|    K|           a11|  2024-01-13|  Type B|
|        20|    T|           a20|  2024-01-22|  Type B|
|         5|    E|            a5|  2024-01-07|  Type B|
|         3|    C|            a3|  2024-01-05|  Type B|
|        12|    L|           a12|  2024-01-14|  Type B|
|        16|    P|           a16|  2024-01-18|  Type B|
|         9|    I|            a9|  2024-01-11|  Type B|
|         4|    D|            a4|  2024-01-06|  Type B|
|         8|    H|            a8|  2024-01-10|  Type B|
|        17|    Q|           a17|  2024-01-19|  Type B|
|        24|    X|           a24|  2024-01-26|  

In [117]:
unioned_df.write.format("csv").option("header","True").save("file:///home/hdoop/notebooks/answer3.csv")

In [136]:
df = unioned_df.join(x_df4,["invoice_no","gstin","supplier_gstin"]).select(col("total_amount").alias("X_amt"),"category")

In [138]:
df2 = unioned_df.join(y_df4,["invoice_no","gstin","supplier_gstin"]).select(col("total_amount").alias("y_amt"),"category")

In [141]:
df3 = df.join(df2,on = ["category"])
df3.show()

+--------+-----+-----+
|category|X_amt|y_amt|
+--------+-----+-----+
|  Type A| 26.4| 15.6|
|  Type A| 15.6| 15.6|
|  Type A| 26.4| 26.4|
|  Type A| 15.6| 26.4|
|  Type B| 28.8| 28.0|
|  Type B| 19.2| 28.0|
|  Type B| 22.8| 28.0|
|  Type B| 37.2| 28.0|
|  Type B| 27.6| 28.0|
|  Type B| 36.0| 28.0|
|  Type B| 38.4| 28.0|
|  Type B| 26.4| 28.0|
|  Type B| 33.6| 28.0|
|  Type B| 39.6| 28.0|
|  Type B| 31.2| 28.0|
|  Type B| 20.4| 28.0|
|  Type B| 15.6| 28.0|
|  Type B| 21.6| 28.0|
|  Type B| 30.0| 28.0|
|  Type B| 25.2| 28.0|
+--------+-----+-----+
only showing top 20 rows



In [144]:
df4 = df3.groupBy("category").agg(
    round(sum("X_amt"),2).alias("total_amount_X"),
    round(sum("y_amt"),2).alias("total_amouunt_Y")
).drop("X_amt").drop("y_amt")

In [147]:
df4.write.format("csv").option("header","True").mode("append").save("file:///home/hdoop/notebooks/answer4.csv")

In [148]:
spark.stop()